In [ ]:
# ============================================================
#   FUZZY MAMDANI - KONTROL KECEPATAN KIPAS ANGIN
# ============================================================

import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import matplotlib.pyplot as plt

# ------------------------------------------------------------
# 1. DEFINISI VARIABEL FUZZY
# ------------------------------------------------------------
suhu = ctrl.Antecedent(np.arange(0, 51, 1), 'Suhu')
kelembaban = ctrl.Antecedent(np.arange(0, 101, 1), 'Kelembaban')
kecepatan = ctrl.Consequent(np.arange(0, 101, 1), 'Kecepatan')

# ------------------------------------------------------------
# 2. FUNGSI KEANGGOTAAN (MEMBERSHIP FUNCTIONS)
# ------------------------------------------------------------

# Fungsi keanggotaan untuk variabel suhu
suhu['Dingin'] = fuzz.trapmf(suhu.universe, [0, 0, 20, 25])
suhu['Normal'] = fuzz.trimf(suhu.universe, [20, 27.5, 35])
suhu['Panas']  = fuzz.trapmf(suhu.universe, [30, 35, 50, 50])

# Fungsi keanggotaan untuk kelembaban
kelembaban['Rendah'] = fuzz.trapmf(kelembaban.universe, [0, 0, 30, 50])
kelembaban['Sedang'] = fuzz.trimf(kelembaban.universe, [30, 50, 70])
kelembaban['Tinggi'] = fuzz.trapmf(kelembaban.universe, [50, 70, 100, 100])

# Fungsi keanggotaan untuk kecepatan kipas
kecepatan['Lambat'] = fuzz.trapmf(kecepatan.universe, [0, 0, 30, 50])
kecepatan['Sedang'] = fuzz.trimf(kecepatan.universe, [30, 50, 70])
kecepatan['Cepat']  = fuzz.trapmf(kecepatan.universe, [50, 70, 100, 100])

# ------------------------------------------------------------
# 3. DEFINISI ATURAN FUZZY (RULE BASE)
# ------------------------------------------------------------
rule1 = ctrl.Rule(suhu['Dingin'], kecepatan['Lambat'])
rule2 = ctrl.Rule(suhu['Normal'] & kelembaban['Rendah'], kecepatan['Lambat'])
rule3 = ctrl.Rule(suhu['Normal'] & kelembaban['Sedang'], kecepatan['Sedang'])
rule4 = ctrl.Rule(suhu['Normal'] & kelembaban['Tinggi'], kecepatan['Sedang'])
rule5 = ctrl.Rule(suhu['Panas'] & kelembaban['Rendah'], kecepatan['Sedang'])
rule6 = ctrl.Rule(suhu['Panas'] & (kelembaban['Sedang'] | kelembaban['Tinggi']), kecepatan['Cepat'])

# ------------------------------------------------------------
# 4. PEMBENTUKAN SISTEM KONTROL DAN SIMULASI
# ------------------------------------------------------------
fan_ctrl = ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5, rule6])
fan_simulasi = ctrl.ControlSystemSimulation(fan_ctrl)

# Input nilai suhu dan kelembaban dari pengguna
input_suhu = float(input("Masukkan Suhu (°C): "))
input_kelembaban = float(input("Masukkan Kelembaban (%): "))

fan_simulasi.input['Suhu'] = input_suhu
fan_simulasi.input['Kelembaban'] = input_kelembaban
fan_simulasi.compute()

# ------------------------------------------------------------
# 5. HASIL DEFUZZIFIKASI DAN OUTPUT SISTEM
# ------------------------------------------------------------
print("\n=== HASIL FUZZY MAMDANI - KONTROL KECEPATAN KIPAS ===")
print(f"Suhu               : {input_suhu} °C")
print(f"Kelembaban         : {input_kelembaban} %")
print(f"Kecepatan Kipas    : {fan_simulasi.output['Kecepatan']:.2f} %")

# ------------------------------------------------------------
# 6. VISUALISASI GRAFIK KEANGGOTAAN (OPSIONAL)
# ------------------------------------------------------------
suhu.view()
kelembaban.view()
kecepatan.view(sim=fan_simulasi)

plt.show()
